In [1]:
import pandas as pd
import numpy as np

In [5]:
#Measures strength of trend
def aroon(df,period=14):
  df['aroon_up']=df['High'].rolling(window=period).apply(lambda x: (x.argmax()-x.argmin())/period,raw=True)
  df['aroon_down']=df['Low'].rolling(window=period).apply(lambda x: (x.argmax()-x.argmin())/period,raw=True)

In [6]:
#Smoothens data for beter trend identification
def heiken_ashi(df):
  df['ha_close']=(df['Open']+df['High']+df['Low']+df['Close'])/4
  df['ha_open']=df['ha_close'].shift(1)
  df['ha_high']=df[['ha_close','ha_open','High']].max(axis=1)
  df['ha_low']=df[['ha_close','ha_open','Low']].min(axis=1)
  df['heiken_ashi']=df.apply(lambda row: 'bullish' if row['ha_close'] > row['ha_open'] else 'bearish', axis=1)
  df.drop(['ha_open','ha_high','ha_low'],axis=1,inplace=True)
  return df

In [4]:
def aroon_heikenashi(df):
  df=heiken_ashi(df)
  df=aroon(df)
  df['signals']=0
  for i in range(1,len(df)):
    if(df['aroon_up'][i]>70 and df['heiken_ashi'][i]=='bullish'):
      df['signals'][i]=1
    elif(df['aroon_down'][i]>70 and df['heiken_ashi'][i]=='bearish'):
      df['signals'][i]=-1
  return df